<a href="https://colab.research.google.com/github/erbisti/Collab/blob/main/Stable_diffusion_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APLICAÇÃO STABLE DIFFUSION (MODELO RNN COM VAE)

### ⚙️ Instalação e ajuste das dependências

Esta etapa prepara o ambiente do **Google Colab** para executar o **Stable Diffusion** com estabilidade e compatibilidade total entre as bibliotecas.

---

#### 🧩 Função de cada pacote

- `Pillow` — é a biblioteca padrão do Python para **manipulação de imagens** (abrir, redimensionar, converter formatos, salvar etc.).  
  O Stable Diffusion a utiliza para ler a imagem de entrada e salvar o resultado final (ex.: `entrada.png` → `saida.png`).  
  A versão deve ser **inferior a 12.0**, pois o **Gradio**, que vem pré-instalado no Colab, ainda não é compatível com as versões mais novas.

- `Gradio` — é um pacote que cria **interfaces gráficas interativas** para modelos de IA diretamente no navegador.  
  Embora o código atual não use Gradio explicitamente, o Colab já o inclui por padrão; por isso é preciso manter o Pillow em uma versão compatível, evitando erros de dependência.  

- `diffusers` — biblioteca da Hugging Face que implementa os **modelos de difusão**, incluindo o **Stable Diffusion**.  
  Ela contém:
  - O **pipeline de geração** (text-to-image, img2img, inpainting etc.);
  - Os **modelos de difusão** baseados em U-Net;
  - E também o **VAE (Variational Autoencoder)** interno, responsável por **codificar e decodificar as imagens no espaço latente** — é nele que o processo de difusão realmente ocorre, tornando o modelo mais leve e eficiente.

- `transformers` — fornece o **modelo de texto** (baseado em CLIP ou Transformer) que converte o prompt em embeddings compreensíveis pela rede de difusão.

- `accelerate` — gerencia automaticamente a **execução em GPU**, garantindo uso ideal da memória e compatibilidade entre dispositivos.

- `safetensors` — formato binário seguro e rápido para **carregar pesos do modelo**, substituindo arquivos `.bin` convencionais.





In [1]:
%pip uninstall -y pillow pillow-simd
%pip install "pillow<12.0"  # compatível com o Gradio do Colab
%pip install -U diffusers transformers accelerate safetensors


Found existing installation: pillow 11.3.0
Uninstalling pillow-11.3.0:
  Successfully uninstalled pillow-11.3.0
  Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.6 MB)


In [2]:
from google.colab import files
up = files.upload()  # escolha o arquivo "entrada.png"


Saving entrada.png to entrada (1).png


### 🧩 O que é o Stable Diffusion

O **Stable Diffusion** é um modelo de **geração de imagens** que combina três componentes principais:  
**VAE**, **rede de difusão (U-Net)** e **modelo de texto (CLIP/Transformer)**.  
Ele pertence à classe dos **Latent Diffusion Models (LDM)**, pois realiza o processo de difusão em um **espaço latente comprimido** — e não diretamente nos pixels da imagem.

---

### 🔸 VAE (Variational Autoencoder)
O **VAE** serve como uma ponte entre imagens e o espaço latente:
- **Encoder**: converte a imagem real (512×512) em um mapa latente menor (ex.: 4×64×64).  
- **Decoder**: reconstrói a imagem final a partir desse espaço latente após a geração.  
Assim, o VAE permite que o modelo trabalhe de forma mais **eficiente** e **compacta**, sem perder os detalhes visuais.

---

### 🔸 Difusão
A **difusão** é o processo central de geração:
1. Parte de um **vetor de ruído** no espaço latente.  
2. Um modelo aprende a **remover o ruído passo a passo** até revelar uma imagem coerente.  
3. Cada passo é guiado por um **prompt textual**, que orienta o conteúdo e o estilo.  
É um processo iterativo de “denoising” controlado por inferência.

---

### 🔸 U-Net
A **U-Net** é a rede responsável por **prever e remover o ruído** em cada etapa da difusão.  
Sua arquitetura em forma de “U” permite combinar **informações locais e globais**:  
as camadas do encoder extraem características gerais, e as do decoder refinam os detalhes visuais.

---

### 🔸 CLIP / Texto-Imagem
O modelo de texto (baseado em **CLIP** ou **Transformer**) **codifica o prompt** em vetores de atenção.  
Esses vetores **guiam a U-Net** durante a difusão, alinhando a geração de imagem com o significado do texto.

---

### 🔸 Pipeline completo


Em resumo, o **Stable Diffusion transforma ruído em imagem coerente** através de:
1. **Codificação textual (CLIP)**,  
2. **Processo de difusão latente (U-Net)**,  
3. **Decodificação visual (VAE)**.  
Esse conjunto é o que torna o modelo capaz de criar imagens detalhadas a partir de descrições naturais.




### 🧠 Codificação textual no Stable Diffusion

O **texto do prompt** é processado por um **modelo de linguagem baseado em Transformer** (no SD 1.5, é o *Text Encoder* do CLIP da OpenAI).  
Esse modelo transforma cada palavra em **vetores numéricos de alta dimensão**, chamados **embeddings** — representações matemáticas do significado das palavras e do contexto.

🔹 **Etapas do processamento textual:**
1. O texto é tokenizado (palavras → índices numéricos).  
2. Esses índices passam por camadas do Transformer, que aprendem **relações semânticas e sintáticas**.  
3. O resultado é uma **matriz de embeddings** com tamanho fixo (ex.: 77×768), representando todo o prompt.

---

### 🧭 Como o texto influencia a geração

Durante o processo de difusão, o **U-Net recebe o vetor latente de imagem (z)** — uma versão comprimida da imagem codificada pelo VAE — e o vetor de texto.  
Por meio de **cross-attention**, o modelo calcula *como cada parte do texto deve afetar cada região do espaço latente*.

🔹 Em outras palavras:
- O **vetor z** representa *como a imagem está sendo formada* (estrutura e ruído).  
- Os **embeddings do texto** dizem *o que essa imagem deve conter*.  
- O mecanismo de **atenção cruzada** faz o texto “modular” o ruído, guiando a U-Net a remover o ruído de forma coerente com o significado das palavras.

---

### 💫 Difusão latente — o núcleo do processo

O **processo de difusão latente** é a sequência de etapas em que o modelo:
1. **Adiciona ruído** gradualmente a imagens latentes durante o treino, até que fiquem irreconhecíveis.  
2. Aprende a **reverter** esse processo — isto é, a remover o ruído passo a passo até recuperar a imagem.  
3. Na geração, parte apenas de ruído puro e executa a sequência inversa (denoising), **guiada pelo texto**.

Tudo isso acontece **no espaço latente do VAE**, e não diretamente nos pixels, o que torna o processo:
- **muito mais leve** (menos memória e cálculos);  
- **mais eficiente** sem perder fidelidade visual.

---

### 🔄 Resumo conceitual

Texto → Embeddings (via Transformer)

↓

Embeddings → Cross-Attention na U-Net

↓

Difusão Latente → remove ruído no vetor z

↓

Decoder VAE → converte z em imagem final


Assim, o **texto molda a trajetória de “limpeza do ruído”** no espaço latente,
fazendo com que o Stable Diffusion **converta ruído em imagens coerentes com a descrição textual.**


In [4]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline, EulerAncestralDiscreteScheduler
from PIL import Image

# Carrega o pipeline em FP16 e envia para GPU
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    dtype=torch.float16,   # <- parâmetro correto para a API atual
    safety_checker=None,
    use_safetensors=True
).to("cuda")

# Scheduler Euler Ancestral
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

# Otimização leve de VRAM (seguro)
pipe.enable_vae_slicing()

# Imagem de entrada (512x512 é nativo do SD 1.5)
img = Image.open("entrada.png").convert("RGB").resize((512, 512), Image.LANCZOS)

# Inferência
out = pipe(
    prompt="(no prompt)",           # opcional: descreva o estilo/cena aqui
    image=img,
    strength=0.40,                  # 0.07–0.15 preserva mais a estrutura; 0.3–0.5 estiliza
    guidance_scale=8.0,
    num_inference_steps=40,
    generator=torch.Generator(device="cuda").manual_seed(365)
)

out.images[0].save("saida1.png")
print("OK")


Keyword arguments {'dtype': torch.float16} are not expected by StableDiffusionImg2ImgPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/16 [00:00<?, ?it/s]

OK


In [5]:
# Veja o resultado com outra semente

img = Image.open("entrada.png").convert("RGB").resize((512, 512), Image.LANCZOS)

out = pipe(
    prompt="(no prompt)",      # ou descreva a cena, se quiser direcionar estilo
    image=img,
    strength=0.40,             # 0.07–0.15 preserva mais a estrutura
    guidance_scale=8.0,
    num_inference_steps=40,
    generator=torch.Generator(device="cuda").manual_seed(42)
)
out.images[0].save("saida2.png")
print("OK")


Keyword arguments {'dtype': torch.float16} are not expected by StableDiffusionImg2ImgPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/16 [00:00<?, ?it/s]

OK


In [6]:
# Veja o resultado com oa mesma semente, mas com alterações de parâmetros

img = Image.open("entrada.png").convert("RGB").resize((512, 512), Image.LANCZOS)

out = pipe(
    prompt="(no prompt)",      # ou descreva a cena, se quiser direcionar estilo
    image=img,
    strength=0.10,             # 0.07–0.15 preserva mais a estrutura
    guidance_scale=8.0,
    num_inference_steps=15, # etapas de filtro
    generator=torch.Generator(device="cuda").manual_seed(42)
)
out.images[0].save("saida3.png")
print("OK")

  0%|          | 0/1 [00:00<?, ?it/s]

OK


In [7]:


img = Image.open("entrada.png").convert("RGB").resize((512,512), Image.LANCZOS)

out = pipe(
    prompt="sharp details, scientific illustration style, change colors",
    negative_prompt="blurry, low contrast, artifacts, text watermark",
    image=img,
    strength=0.4,             # preserva a estrutura original
    guidance_scale=12.0,      # este parâmetro serve para dar 'força' ao texto
    num_inference_steps=40,
    generator=torch.Generator(device="cuda").manual_seed(42)
)
out.images[0].save("saida4.png")


  0%|          | 0/16 [00:00<?, ?it/s]

In [8]:
img = Image.open("entrada.png").convert("RGB").resize((512, 512), Image.LANCZOS)

out = pipe(
    prompt="deform image",
    negative_prompt="low contrast, monocolor",
    image=img,
    strength=0.80,            # altíssimo valor, não preservará a estrutura original
    guidance_scale=12.0,
    num_inference_steps=40,
    generator=torch.Generator(device="cuda").manual_seed(1000)
)
out.images[0].save("saida5.png")


  0%|          | 0/32 [00:00<?, ?it/s]

### Vamos entrar com uma paisagem para ver os efeitos que o stable diffusion pode fazer!


In [9]:
from google.colab import files
up = files.upload()  # escolha o arquivo "bonita.png"


Saving bonita.jpg to bonita.jpg


In [10]:
# Imagem de entrada (512x512 é nativo do SD 1.5)
img = Image.open("bonita.jpg").convert("RGB").resize((506, 900), Image.LANCZOS)

# Inferência
out = pipe(
    prompt="(no prompt)",           # opcional: descreva o estilo/cena aqui
    image=img,
    strength=0.40,                  # 0.07–0.15 preserva mais a estrutura; 0.3–0.5 estiliza
    guidance_scale=8.0,
    num_inference_steps=40,
    generator=torch.Generator(device="cuda").manual_seed(365)
)

out.images[0].save("saida_bonita1.png")
print("OK")


  0%|          | 0/16 [00:00<?, ?it/s]

OK


In [11]:
# Imagem de entrada (512x512 é nativo do SD 1.5)
img = Image.open("bonita.jpg").convert("RGB").resize((506, 900), Image.LANCZOS)

# Inferência
out = pipe(
    prompt="darkness day, raining, lightining and thunders",           # opcional: descreva o estilo/cena aqui
    image=img,
    strength=0.80,                  # 0.07–0.15 preserva mais a estrutura; 0.3–0.5 estiliza
    guidance_scale=12.0,
    num_inference_steps=45,
    generator=torch.Generator(device="cuda").manual_seed(365)
)

out.images[0].save("saida_bonita2.png")
print("OK")


  0%|          | 0/36 [00:00<?, ?it/s]

OK
